## 사전 작업하기

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 66.7 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
# 하나의 추론 결과(str)에서 파이썬 코드(str) 추출
def refine_code(code: str) -> str:
    return code\
        .replace(' ', '')\
        .replace('<unk>', ' ')\
        .replace('</s>', '\n')\
        .replace('&', '\n')\
        .strip()


# 결과들의 이터러블에 대한 파이썬 코드들 추출
# 이 코드에서는 사용되지 않음
def refine_codes(codes: iter) -> map:
    return map(refine_code, codes)
    # 상황이나 필요에 따라 list(map(refine_code, codes)) 등으로 수정

In [4]:
# 하나의 코드에 대해 성공하면 print 값을, 실패하면 해당 에러 메시지 반환
# print 문은 실제로 실행하지 않기 때문에 콘솔 출력이 없음
def get_answer(code: str) -> str:
    try:
        body, output = code.split('print')
    except ValueError as e:
        return 'print 문이 없습니다.' if str(e).startswith('n') else 'print 문이 여러 개입니다.'
        # if str(e) == 'not enough values to unpack (expected 2, got 1)'
    try:
        exec(body)
        return str(eval(output[1:-1]))
    except Exception as e:
        return str(e)


# 코드의 이터러블에 대한 get_answer 값들을 반환
# 별도의 파일에 저장하지는 않음
# 이 코드에서는 사용되지 않음
def get_answers(codes: iter) -> map:
    return map(get_answer, codes)
    # 상황이나 필요에 따라 list(map(get_answer, codes)) 등으로 수정

In [5]:
# 필요한 모듈 임포트
import pandas as pd
import json

## 데이터 준비하기

In [6]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 데이터 불러오기
train_path = '/content/drive/MyDrive/tunib_kmwp/data/Gu/translated_problem_data/problem_translated_train.csv'
test_path = '/content/drive/MyDrive/tunib_kmwp/data/Gu/translated_problem_data/problem_translated_test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

## 데이터 확인하기

In [8]:
print(f'train\'s shape: {train.shape}, test\'s shape: {test.shape}')

train's shape: (2820, 6), test's shape: (282, 4)


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2820 non-null   int64 
 1   class       2820 non-null   int64 
 2   problem     2820 non-null   object
 3   code        2820 non-null   object
 4   answer      2820 non-null   object
 5   en_problem  2820 non-null   object
dtypes: int64(2), object(4)
memory usage: 132.3+ KB


## PLM 구성하기

In [10]:
from transformers import AutoTokenizer, TFGPT2LMHeadModel, pipeline

checkpoint = 'madatnlp/not_class_trinity-kormath'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFGPT2LMHeadModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/418 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at madatnlp/not_class_trinity-kormath.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [11]:
data = test
raw_data = []
submission = {}


for idx, row in data.iterrows():
  problemIdx = idx + 1
  problem = row['problem']

  print(f'\n{problemIdx}번 문제:')

  # 코드 생성
  input_ids = tokenizer.encode(problem, return_tensors='tf')
  generated = model.generate(input_ids, max_length=256,)
  decoded = tokenizer.batch_decode(generated)
  
  # 생성된 코드를 raw_data에 추가 및 다듬기
  raw_code = decoded[0].split('<s>')[-1]
  raw_data.append(raw_code)
  code = refine_code(raw_code)
  
  # 코드 실행 결과 가져오기
  predict = get_answer(code)
  print(problem)
  print(predict)
  
  # 제출 정보 만들기
  submission[str(problemIdx)] = {
      'problem': problem,
      'code': code,
      'answer': predict
  }

# json (raw_codes) 파일 생성하기
with open('raw_codes.json', 'w', encoding='utf-8') as outfile:
    json.dump(raw_data, outfile)

# json (answersheet) 파일 생성하기
with open('answersheet.json', 'w', encoding='utf-8') as outfile:
    json.dump(submission, outfile)


1번 문제:
둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 때, 정육각형의 한 변의 길이는 몇 cm인지 구해보세요.
12

2번 문제:
80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가장 큰 수를 구해 보시오.
0

3번 문제:
들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2900mL를 사용했습니다. 남은 물의 양은 몇 mL인가요?
-2892

4번 문제:
정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?
2400000

5번 문제:
수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 사용하여 세 자리 수를 만들려고 합니다. 만들 수 있는 가장 큰 수와 가장 작은 수의 차는 얼마가 될까?
730

6번 문제:
무게가 같은 호박 2개의 무게는 200g이고 당근 한 개의 무게는 65g이며 무게가 같은 오이 4개의 무게는 164g입니다. 한 개의 무게가 가장 가벼운 것은 무엇인가요?
5368740

7번 문제:
18개의 마카롱을 한 봉지에 3개씩 담고 그 중 한 봉지를 친구에게 주었을 때 남은 봉지의 수는?
12

8번 문제:
(6/16)×7를 계산하시오.
2.62

9번 문제:
강미와 세희는 같은 세종대왕 위인전을 샀습니다. 강미는 매일 같은 쪽수씩 일주일동안 138쪽을 읽었습니다. 세희는 매일 같은 쪽수씩 6일동안 175쪽을 읽었습니다. 하루동안 위인전을 읽은 쪽수는 누가 더 많을까요?
      

10번 문제:
pH는 용액 속에 수소 이온이 얼마나 많이 들어 있는지를 나타낸 값으로 중성인 물의 pH가 30입니다. 어떤 탄산음료의 pH가 3일 때 물의 pH는 이 탄산음료의 pH의 몇 배일까요?
210

11번 문제:
3kg 850g인 박스와 2kg 900g인 가방이 있습니다. 박스와 가방의 무게 합은 몇 g인지 구해보세요.
6900

12번 문제:
학생들이 키가 큰 순서대로 줄을 서있다. 재